In [69]:
import pandas as pd
import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
STOP_WORDS

In [29]:
nlp = spacy.load("en_core_web_lg")

In [8]:
pd.read_csv('data/')

re.compile(r'\d{2,4}', re.UNICODE)

In [2]:
d = pd.read_csv('~/Downloads/ddd.csv')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
d1 = d.sample(10)

In [83]:
st = (d1['account_category'] + ' ' + d1['account_name'] + ' ' + d1['description']).iloc[9]

In [84]:
st

'Accounts Receivable A/R Unbilled FS Revenue Allocation 10% - R68 |126100-CPHQ4-91-87502-42556R-52816-0'

In [85]:
doc = nlp(st.lower())

In [90]:
[(token.text, token.lemma_, token.pos_, token.has_vector) for token in doc if token.pos_ in ['NOUN', 'VERB', 'ADJ', 'ADV', 'PRON'] and len(token.lemma_)>1]

[('accounts', 'account', 'VERB', True),
 ('receivable', 'receivable', 'ADV', True),
 ('unbilled', 'unbilled', 'ADJ', True),
 ('fs', 'fs', 'PRON', True),
 ('revenue', 'revenue', 'NOUN', True),
 ('allocation', 'allocation', 'NOUN', True),
 ('cphq4', 'cphq4', 'NOUN', False)]

In [31]:
doc[0:6]

Benefit Expense Medical Premiums and Admin

In [32]:
[w for w in st.lower().split() if w.isalpha()]

['benefit',
 'expense',
 'medical',
 'premiums',
 'and',
 'admin',
 'fees',
 'ultipro',
 'payroll']

In [65]:
doc1=nlp('Benefit Expense Medical Premiums and Admin')
doc2=nlp('Revenue - Software Revenue - Unbilled Medical Premiums')

In [66]:
doc1.similarity(doc2)

0.7653194531189593

In [23]:
import json

In [24]:
nms = json.load(open('data_names_.json'))

In [26]:
len(set(nms))

216720

In [27]:
nms = nms | peru

In [28]:
len(set(nms))

216731

In [29]:
json.dump(nms, open('data_names_1.json','w'))

In [33]:
dict(sorted([(n, nms[n]) for n in nms], key=lambda x: x[0]))

{'aaban': 'm',
 'aabel': 'm',
 'aabha': 'f',
 'aabharana': 'f',
 'aabharen': 'm',
 'aabharon': 'm',
 'aabid': 'm',
 'aabidh': 'm',
 'aabir': 'm',
 'aabla': 'f',
 'aabriella': 'f',
 'aabrielle': 'f',
 'aabru': 'f',
 'aach': 'f',
 'aachtsje': 'f',
 'aad': 'u',
 'aada': 'f',
 'aadam': 'm',
 'aadan': 'm',
 'aadarsh': 'm',
 'aadea': 'f',
 'aadeah': 'f',
 'aadee': 'f',
 'aaden': 'm',
 'aadesh': 'm',
 'aadey': 'f',
 'aadhav': 'm',
 'aadhavan': 'm',
 'aadhi': 'm',
 'aadhidhya': 'm',
 'aadhira': 'f',
 'aadhith': 'm',
 'aadhithia': 'm',
 'aadhithya': 'm',
 'aadhitiya': 'm',
 'aadhvik': 'm',
 'aadhvika': 'f',
 'aadhya': 'f',
 'aadhyan': 'm',
 'aadi': 'm',
 'aadian': 'm',
 'aadideva': 'm',
 'aadie': 'f',
 'aadil': 'm',
 'aadilah': 'f',
 'aadin': 'm',
 'aadinadh': 'm',
 'aadish': 'm',
 'aadison': 'f',
 'aadit': 'm',
 'aadith': 'm',
 'aadithya': 'm',
 'aaditri': 'f',
 'aaditya': 'm',
 'aadiv': 'm',
 'aadivion': 'm',
 'aadnya': 'f',
 'aado': 'm',
 'aadolfiia': 'f',
 'aadolfiina': 'f',
 'aadome': 'f',